In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from pathlib import Path
from selenium import webdriver
import datetime
from loguru import logger
import datetime
import time
import random
import matplotlib.pyplot as plt
import numpy as np
import os
from loguru import logger

In [54]:
# a logging support time difference class
class logtimediff():
    
    def __init__(self,name=None):
        self.start=datetime.datetime.now()
        self.reference={}
        if name is not None:
            self.ref(name)
#             print(name)

        
#     def diff(self):
#         name=None
#         if name is not None:
#             return datetime.datetime.now() - self.ref['name']
#         else:
#             return datetime.datetime.now() - self.start
        
    
    def ref(self,name=None):
        if name is not None:
            print(name)
            self.reference[name]=datetime.datetime.now()
        
        
a=logtimediff('mystart')
# a

mystart


In [81]:
home_url='https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1'
home_url='https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220'
#          'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1'
#     'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/cozy-studio-near-schmiede-wiedikon-1405chf-month.2072688?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1

logger.info('starting scraping')
driver = webdriver.Firefox(executable_path=Path(r'C:\Users\Bo-user\Downloads\geckodriver-v0.27.0-win64') / 'geckodriver.exe')
driver.get(home_url)
previews=[]
maxiter = 10
last=0
for i in range(maxiter):
    # iterate the scrolldown move until we found the buton to the next index page
    last= wait_minimum(abs(random.gauss(1, 1)) + 3,last)
   
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    logger.info(f'{i} 1')
    content = driver.page_source # this is one big string of webpage html
    soup = BeautifulSoup(content)
    logger.info(f'{i} 2')
    goal = soup.find('a',attrs={'class': 'pages_links_href pages_arrow'})
    if goal:
        break
content = driver.page_source # this is one big string of webpage html
soup = BeautifulSoup(content)
logger.info('soup made')


2020-10-18 23:10:04.031 | INFO     | __main__:<module>:6 - starting scraping
2020-10-18 23:10:14.694 | INFO     | __main__:<module>:17 - 0 1
2020-10-18 23:10:15.477 | INFO     | __main__:<module>:20 - 0 2
2020-10-18 23:10:18.500 | INFO     | __main__:<module>:17 - 1 1
2020-10-18 23:10:19.195 | INFO     | __main__:<module>:20 - 1 2
2020-10-18 23:10:23.600 | INFO     | __main__:<module>:17 - 2 1
2020-10-18 23:10:24.284 | INFO     | __main__:<module>:20 - 2 2
2020-10-18 23:10:27.775 | INFO     | __main__:<module>:17 - 3 1
2020-10-18 23:10:28.528 | INFO     | __main__:<module>:20 - 3 2
2020-10-18 23:10:33.334 | INFO     | __main__:<module>:17 - 4 1
2020-10-18 23:10:34.073 | INFO     | __main__:<module>:20 - 4 2
2020-10-18 23:10:39.011 | INFO     | __main__:<module>:17 - 5 1
2020-10-18 23:10:39.911 | INFO     | __main__:<module>:20 - 5 2
2020-10-18 23:10:40.624 | INFO     | __main__:<module>:26 - soup made


In [64]:
import time
a= time.time()
time.sleep(1)
b= time.time()
b-a

1.0000061988830566

In [78]:
def wait_minimum(f=0,last=None,):
    if last:
        now = time.time()
        sleeptime = max(f -(now-last),0)
        time.sleep(sleeptime)
    return  time.time()
    
    
    
last = wait_minimum()
print(last)
time.sleep(2)
wait_minimum(1,last)

1603055269.479236


1603055271.480111

In [83]:
datetime.datetime(0,0,0,0,0,0)

ValueError: year 0 is out of range

In [87]:
[element for element in soup.find_all('a',attrs={'class': 'pages_links_href pages_arrow'}) if element['title'] =='Weiter']


[<a class="pages_links_href pages_arrow" data-category="Pager" data-label="Bottom" data-value="7" href="/zuerich/immobilien/wg-zuerich.1220?sort=created&amp;page=7&amp;hash=8066eefae06f4ebdde76511cbb811cdf" onclick="tracked_event_handler.call(this, {type:'next'});" rel="next" title="Weiter">&gt;</a>]

In [82]:
from urllib.parse import urlparse
urlparse(home_url).netloc+soup.find('a',attrs={'class': 'pages_links_href pages_arrow'})['href']


'www.ronorp.net/zuerich/immobilien/wg-zuerich.1220?sort=created&page=5&hash=8066eefae06f4ebdde76511cbb811cdf'

In [41]:
b=datetime.datetime.fromtimestamp(os.stat(a).st_mtime)
a=datetime.datetime.now()

In [44]:
def time_difference(start,end):
    diff = (end - start)
    return diff.seconds



0.0636111111111111

In [5]:
a=np.array([np.random.random(5) for i in range(3)])
a

array([[0.73894443, 0.4805539 , 0.3970481 , 0.21233343, 0.72104877],
       [0.20293738, 0.23233509, 0.21945163, 0.99453118, 0.51079273],
       [0.87874704, 0.65045306, 0.81190077, 0.95994057, 0.76553608]])

In [6]:
pd.DataFrame(a)

,0,1,2,3,4
0,0.738944,0.480554,0.397048,0.212333,0.721049
1,0.202937,0.232335,0.219452,0.994531,0.510793
2,0.878747,0.650453,0.811901,0.959941,0.765536
